In [15]:
from flo_ai import Flo
from flo_ai import FloSession
from langchain_openai import ChatOpenAI

from dotenv import load_dotenv
load_dotenv()

True

# Setting up the send email tool

In [8]:
from typing import Optional, Type
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)

class SendEmailInput(BaseModel):
    email: str = Field(description="The email text to be sent")

class SendEmailTool(BaseTool):
    name = "email_triage"
    description = "useful for when you need to send an email to someone"
    args_schema: Type[BaseModel] = SendEmailInput

    def _run(
        self, email: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        print(email)
        return "Email sent successfully"

    async def _arun(
        self, email: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        print(email)
        return "Email sent successfully"

# Setting up the fetch transaction tool

In [9]:
from typing import Optional, Type
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)

class FetchTrxInput(BaseModel):
    reference_number: str = Field(description="The transaction reference number")

class FetchTransactionTool(BaseTool):
    name = "fetch_transactions"
    description = "useful for when you want to fetch the transaction details given reference number"
    args_schema: Type[BaseModel] = FetchTrxInput

    def _run(
        self, reference_number: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        return "The transaction happened on 23/07/2024 IST and it failed because there was not enough balance in the account"

    async def _arun(
        self, reference_number: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        return "The transaction happened on 23/07/2024 IST and it failed because there was not enough balance in the account"

# Register the tools

In [12]:
llm = ChatOpenAI(temperature=0, model_name='gpt-4o')
session = FloSession(llm).register_tool(
    name="SendEmailTool", 
    tool=SendEmailTool()
).register_tool(
    name="FetchTransactionTool",
    tool=FetchTransactionTool()
)

# Setting up the agentic flo

In [19]:
agent_yaml = """
apiVersion: flo/alpha-v1
kind: FloRoutedTeam
name: support-email-handler
team:
    name: SupportTicketHandler
    router:
        name: SupportSupervisor
        kind: supervisor
    agents:
      - name: EmailSender
        prompt: You are capable of sending the reply email but constructing a apt response
        tools:
          - name: SendEmailTool
      - name: TransactionFetcher
        prompt: You are capable of fetching any kind of transactions from the database given transaction reference id
        tools:
          - name: FetchTransactionTool
"""

# Setup input prompt

In [28]:
email_input = """
Hi Tom,

We have a failed transaction. Can you tell me more about why this failed.

Thanks
Vishnu
"""

In [29]:
input_prompt = """
You are a support ticket handling agent. You are expected to understand the email, and perform the following steps:

1. Reply asking for more information if there is missing information in the email
2. Perform the neccessary steps to required to answer the users question and reply the user query.
3. If are not able to come up with a answer just reply I dont know over the email

Email:

""" + email_input

## Run the flo

In [30]:
flo: Flo = Flo.build_with_yaml(session, yaml=agent_yaml)
for s in flo.stream(input_prompt):
     if "__end__" not in s:
        print(s)
        print("----")

{'SupportSupervisor-x7llc': {'next': 'TransactionFetcher-YdHZo'}}
----


> Entering new AgentExecutor chain...
Hi Vishnu,

Thank you for reaching out.

Could you please provide me with the transaction reference number for the failed transaction? This will help me to look into the details and provide you with the necessary information.

Looking forward to your response.

Best regards,  
Tom

> Finished chain.
{'TransactionFetcher-YdHZo': {'messages': [HumanMessage(content='Hi Vishnu,\n\nThank you for reaching out.\n\nCould you please provide me with the transaction reference number for the failed transaction? This will help me to look into the details and provide you with the necessary information.\n\nLooking forward to your response.\n\nBest regards,  \nTom', name='TransactionFetcher-YdHZo')]}}
----
{'SupportSupervisor-x7llc': {'next': 'FINISH'}}
----
